In [18]:
# Dependencies
import json
from pprint import pprint
from datetime import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tweepy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from config import (consumer_key, consumer_secret,
                    access_token, access_token_secret)

In [19]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), 
                 wait_on_rate_limit=False, wait_on_rate_limit_notify=False)

# Initialize Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
new_list ["#Gringo", "#TheHurricantHeist", "#StrangersPreyAtNight", "#AWrinkleinTime", "#LoveSimon", "#TombRaider",
          "#PacificRimUprising","#SherlockGnomes","#TylerPerrysAcrimony","#ReadyPlayerOne"]

In [20]:
# Target user
search_term = "tomb raider (2018)"

# Create variable for holding the oldest tweet
oldest_tweet = None

# Create list of dictionaries
sentiment = []

# "Real Person" Filters
min_tweets = 5
max_tweets = 10000
max_followers = 2500
max_following = 2500
lang = "en"

# Date
search_date = "2018-03-12"

# try:

# Loop through 50 times (total of 50000 tweets)
for x in range(50):
    
    # Get all tweets from home feed (for each page specified)
    public_tweets = api.search(search_term, 
                                count=100,
                                lang='en', 
                                until=search_date, 
                                max_id=oldest_tweet)

    # Loop through all tweets
    for tweet in public_tweets['statuses']:
        
        # Use filters to check if user meets conditions
        if (tweet["user"]["followers_count"] < max_followers and
            tweet["user"]["statuses_count"] > min_tweets and
            tweet["user"]["statuses_count"] < max_tweets and
            tweet["user"]["friends_count"] < max_following and
            tweet["user"]["lang"] == lang):
        
            # Grab tweet data
            name = tweet['user']['screen_name']
            tweet_text = tweet['text']
            date = tweet['created_at']
            
            # Run Vader Analysis on each tweet
            results = analyzer.polarity_scores(tweet["text"])
            compound = results["compound"]
            positive = results['pos']
            neutral = results['neu']
            negative = results['neg']
            
            # Create dictionary holding tweet data
            tweet_dict = {'Handle': name, 'Tweet': tweet_text, 'Date': date, 'Compound': compound, 
                          'Positive': positive, 'Neutral': neutral, 'Negative': negative}
    
            # Append tweet data to sentiment list
            sentiment.append(tweet_dict)
    
            # Reassign the the oldest tweet (i.e. the max_id)
            oldest_tweet = int(tweet["id_str"])
        
            # Subtract 1 so the previous oldest isn't included
            # in the new search
            oldest_tweet = oldest_tweet - 1

# except:
#     print("You exceeded the rate limit!")
#     raise

TweepError: Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/search/tweets.json?q=tomb+raider+%282018%29&count=1000&lang=en&until=2018-03-12&max_id=971709022507188223 (Caused by SSLError(SSLError("bad handshake: SysCallError(10054, 'WSAECONNRESET')",),))

In [21]:
# Create dataframe
awit_tweets = pd.DataFrame(sentiment)

# Reorder columns
awit_tweets = awit_tweets.iloc[:,[2,6,1,0,5,4,3]]

# Save to csv
awit_tweets.to_csv(f'sent_data/{search_date}_TR_sentiment.csv', encoding='utf-8', index=False)
awit_tweets

,Handle,Tweet,Date,Compound,Positive,Neutral,Negative
0,vinnymoon,RT @gameinformer: The first three Tomb Raider ...,Sun Mar 11 23:56:48 +0000 2018,0.5106,0.136,0.864,0.000
1,JeddtheJedi,Here's my review of #TombRaider - I enjoyed it...,Sun Mar 11 23:56:08 +0000 2018,0.3818,0.223,0.661,0.116
2,VikvnderAlicia,"RT @TombRaiderMovie: In 4 DAYS, discover how s...",Sun Mar 11 23:53:39 +0000 2018,0.0000,0.000,1.000,0.000
3,horror_returns,"RT @TombRaiderMovie: In 4 DAYS, discover how s...",Sun Mar 11 23:48:57 +0000 2018,0.0000,0.000,1.000,0.000
4,nclh_chong,"RT @TombRaiderMovie: In 4 DAYS, discover how s...",Sun Mar 11 23:47:42 +0000 2018,0.0000,0.000,1.000,0.000
5,TombRaiderArch,"RT @TombRaiderMovie: In 4 DAYS, discover how s...",Sun Mar 11 23:43:08 +0000 2018,0.0000,0.000,1.000,0.000
6,patrickvons,"Disney could not unseat Marvel, with ""A Wrinkl...",Sun Mar 11 23:40:52 +0000 2018,-0.5472,0.000,0.815,0.185
7,dirk_pinter,"RT @TombRaiderMovie: In 4 DAYS, discover how s...",Sun Mar 11 23:38:28 +0000 2018,0.0000,0.000,1.000,0.000
8,jgyabzz,"RT @TombRaiderMovie: In 4 DAYS, discover how s...",Sun Mar 11 23:30:06 +0000 2018,0.0000,0.000,1.000,0.000
9,ChasePond64,@RayNicolet Here's some examples silly!: https...,Sun Mar 11 23:29:04 +0000 2018,0.0000,0.000,1.000,0.000


In [ ]:
# Check duplicate tweets
# awit_tweets[awit_tweets['Tweet'].duplicated(keep='first')]